In [1]:
import re
import time
from colorama import Fore
import openpyxl
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import random
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
import math

In [2]:
options = Options()

options.add_argument('--disable-application-cache')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--disable-cookies")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.add_argument("start-maximized")
options.add_argument('--disable-gpu')
# options.add_argument("--headless")  
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument('--ignore-urlfetcher-cert-requests')
options.add_argument('--no-first-run')
options.add_argument("--disable-popup-blocking") 


driver = webdriver.Chrome(options=options)

NameError: name 'Options' is not defined

In [3]:
link='https://str.optical.org/'

In [4]:
driver.get(link)

In [5]:
england_postcode_areas = [
'NE','YO','DL','CA','MI','S','B','L','E','N','TA','EX'
]

In [ ]:
# england_postcode_areas = [
# 'DL','CA','MI','S','B','L','E','N','TA','EX'
# ]

In [6]:
inputcodepostal=driver.find_element(By.ID,'Registrant-Geography-input')
clicksearch=driver.find_element(By.CSS_SELECTOR,'#searchform > div.card-footer > button.btn.btn-primary.float-right.mt-3')

In [7]:
slider = driver.find_element(By.ID, "BodyCorporate-Geography-range")

# Ejecuta JS para ponerlo en 100 y disparar el evento
driver.execute_script("""
    arguments[0].value = 100;
    arguments[0].dispatchEvent(new Event('input', { bubbles: true }));
""", slider)



In [8]:
data1=[]
data2=[]
data3=[]
data4=[]
urls=[]

In [9]:
rango = [5]

for area in england_postcode_areas:
    for district in rango:
        retries = 3
        while retries > 0:
            try:
           
                slider = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, "Registrant-Geography-range"))
                )
                inputcodepostal = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, 'Registrant-Geography-input'))
                )
                clicksearch = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, '#searchform > div.card-footer > button.btn.btn-primary.float-right.mt-3'))
                )

                inputcodepostal.clear()
                inputcodepostal.send_keys(area + str(district))

  
                driver.execute_script("""
                    arguments[0].value = 100;
                    arguments[0].dispatchEvent(new Event('input', { bubbles: true }));
                """, slider)

   
                clicksearch.click()
                time.sleep(.5)

         
                soup1 = BeautifulSoup(driver.page_source, 'html.parser')
                prepage = soup1.find('p', class_='d-block text-right mt-4')
                if not prepage:
                    raise ValueError("No se encontró el texto de número de páginas.")

                maxpage = int(prepage.text.strip().split()[-1])

                for numpage in range(1, maxpage + 1):
                    try:
                        print(numpage)
                        inputpag = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.ID, 'goToPageInput'))
                        )
                        inputpag.clear()
                        inputpag.send_keys(numpage)
                        time.sleep(.5)

                        searchbutton = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, '#search-results > div.card-body > div:nth-child(3) > div.col-md-4 > div > div > button'))
                        )
                        searchbutton.click()
                        time.sleep(1)

       
                        soup2 = BeautifulSoup(driver.page_source, 'html.parser')
                        alltargets = soup2.find_all('li', class_='media mb-3 border-bottom pb-3')

                        for target in alltargets:
                            try:
                                predata1 = target.find('strong')
                                data1.append(predata1.text if predata1 else None)
                            except:
                                data1.append(None)
                            
                            
                            try:
                                predata2 = target.find('div', class_='media-body')
                                preurl=target.find('a', class_='btn btn-primary')['href'] if predata2 else None
                                if preurl:
                                    preurl = 'https://str.optical.org' + preurl
                                    urls.append(preurl)
                                if predata2:
                                    lines = predata2.text.strip().split('\n')
                                    if lines:
                                        data2.append(lines[len(lines)-1].strip())
                                    else:
                                        data2.append(None)
                                else:
                                    data2.append(None)
                            except:
                                data2.append(None)
                            
                            
                            try:
                                predata3 = target.find('div', class_='media-body')
                                if predata3:
                                    lines = predata3.text.strip().split('\n')
                                    if lines:
                                        data3.append(lines[len(lines)-2].strip())
                                    else:
                                        data3.append(None)
                                else:
                                    data3.append(None)
                            except:
                                data3.append(None)
                            try:
                                predata4 = target.find('div', class_='media-body')
                                if predata4:
                                    lines = predata4.text.strip().split('\n')
                                    if lines:
                                        data4.append(lines[len(lines)-3].strip())
                                    else:
                                        data4.append(None)
                                else:
                                    data4.append(None)
                            except:
                                data4.append(None)
                    except Exception as e:
                        print(f"[!] Error en paginación página {numpage}: {e}")
                        continue
                break  
            except Exception as e:
                print(f"[!] Error con {area}{district}, reintentando... {3 - retries + 1}/3 - {e}")
                retries -= 1
                time.sleep(.5)
        else:
            print(f"[x] Fallo definitivo en {area}{district}")

[!] Error con NE5, reintentando... 1/3 - No se encontró el texto de número de páginas.
1
[!] Error en paginación página 1: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=137.0.7151.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff76048cda5+78885]
	GetHandleVerifier [0x0x7ff76048ce00+78976]
	(No symbol) [0x0x7ff760249bca]
	(No symbol) [0x0x7ff7602515bd]
	(No symbol) [0x0x7ff76025460c]
	(No symbol) [0x0x7ff7602546df]
	(No symbol) [0x0x7ff7602a168a]
	(No symbol) [0x0x7ff760296ffc]
	(No symbol) [0x0x7ff7602c8b8a]
	(No symbol) [0x0x7ff760292f06]
	(No symbol) [0x0x7ff7602c8da0]
	(No symbol) [0x0x7ff7602f122f]
	(No symbol) [0x0x7ff7602c8963]
	(No symbol) [0x0x7ff7602916b1]
	(No symbol) [0x0x7ff760292443]
	GetHandleVerifier [0x0x7ff760764eed+3061101]
	GetHandleVerifier [0x0x7ff76075f33d+30376

KeyboardInterrupt: 

In [47]:
sample=({'Name':data1,
         'Adress':data2,
         'Speciality':data3,
         'Register_as':data4,
            'Url':urls
         

})

In [48]:
sample=pd.DataFrame(sample)

In [49]:
sample.drop_duplicates(inplace=True)

In [13]:
sample['Personal Adress']=None

In [14]:
sample['Town']=None

In [ ]:
for index,e in enumerate(sample['Url']):
    driver.get(e)
    time.sleep(1)
    soup3 = BeautifulSoup(driver.page_source, 'html.parser')

    try:
        preTown = soup3.find('div', attrs={'aria-label': 'Town', 'class': 'media-body'}).text.replace('\n', ' ').strip()
        if preTown:
            sample['Town'][index] = preTown

    except AttributeError:
        print(f"[!] Error al obtener la ciudad personal para {e}: {e}")
    try:
        prePersonalAdress = soup3.find('div', attrs={'aria-label': 'Practice addresses', 'class': 'media-body'}).text.replace('\n', ' ').strip()
        if prePersonalAdress:
            sample['Personal Adress'][index+3000] = prePersonalAdress
    except AttributeError:
        print(f"[!] Error al obtener la dirección personal para {e}: {e}")
    
        

C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4173164883.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sample['Town'][index+3000] = preTown
C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4173164883.py:16: FutureWarning: ChainedAss

[!] Error al obtener la ciudad personal para https://str.optical.org/Registrant/4874F67C-DC49-EB11-A812-000D3AD68B6C?filterId=Registrant: https://str.optical.org/Registrant/4874F67C-DC49-EB11-A812-000D3AD68B6C?filterId=Registrant
[!] Error al obtener la dirección personal para https://str.optical.org/Registrant/4874F67C-DC49-EB11-A812-000D3AD68B6C?filterId=Registrant: https://str.optical.org/Registrant/4874F67C-DC49-EB11-A812-000D3AD68B6C?filterId=Registrant


C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4173164883.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sample['Town'][index+3000] = preTown
C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4173164883.py:16: FutureWarning: ChainedAss

[!] Error al obtener la ciudad personal para https://str.optical.org/Registrant/87BFDAF5-D149-EB11-A812-000D3AD68B6C?filterId=Registrant: https://str.optical.org/Registrant/87BFDAF5-D149-EB11-A812-000D3AD68B6C?filterId=Registrant
[!] Error al obtener la dirección personal para https://str.optical.org/Registrant/87BFDAF5-D149-EB11-A812-000D3AD68B6C?filterId=Registrant: https://str.optical.org/Registrant/87BFDAF5-D149-EB11-A812-000D3AD68B6C?filterId=Registrant


C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4173164883.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sample['Town'][index+3000] = preTown
C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4173164883.py:16: FutureWarning: ChainedAss

...

C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4173164883.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sample['Town'][index+3000] = preTown
C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4173164883.py:16: FutureWarning: ChainedAss

[!] Error al obtener la ciudad personal para https://str.optical.org/Registrant/42FD55A2-C149-EB11-A812-00224801C499?filterId=Registrant: https://str.optical.org/Registrant/42FD55A2-C149-EB11-A812-00224801C499?filterId=Registrant
[!] Error al obtener la dirección personal para https://str.optical.org/Registrant/42FD55A2-C149-EB11-A812-00224801C499?filterId=Registrant: https://str.optical.org/Registrant/42FD55A2-C149-EB11-A812-00224801C499?filterId=Registrant


C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4173164883.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sample['Town'][index+3000] = preTown
C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4173164883.py:16: FutureWarning: ChainedAss

[!] Error al obtener la ciudad personal para https://str.optical.org/Registrant/6899F3E7-E449-EB11-A812-000D3AD68B6C?filterId=Registrant: https://str.optical.org/Registrant/6899F3E7-E449-EB11-A812-000D3AD68B6C?filterId=Registrant
[!] Error al obtener la dirección personal para https://str.optical.org/Registrant/6899F3E7-E449-EB11-A812-000D3AD68B6C?filterId=Registrant: https://str.optical.org/Registrant/6899F3E7-E449-EB11-A812-000D3AD68B6C?filterId=Registrant
[!] Error al obtener la ciudad personal para https://str.optical.org/Registrant/2F5D3C6A-C149-EB11-A812-000D3AD68B6C?filterId=Registrant: https://str.optical.org/Registrant/2F5D3C6A-C149-EB11-A812-000D3AD68B6C?filterId=Registrant
[!] Error al obtener la dirección personal para https://str.optical.org/Registrant/2F5D3C6A-C149-EB11-A812-000D3AD68B6C?filterId=Registrant: https://str.optical.org/Registrant/2F5D3C6A-C149-EB11-A812-000D3AD68B6C?filterId=Registrant
[!] Error al obtener la ciudad personal para https://str.optical.org/Regis

C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4173164883.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sample['Town'][index+3000] = preTown
C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4173164883.py:16: FutureWarning: ChainedAss

In [29]:
for index,e in enumerate(sample['Personal Adress']):
    

    if e is None:
            link = sample['Url'][index]
            driver.get(link)
            time.sleep(1)
            soup3 = BeautifulSoup(driver.page_source, 'html.parser')
            try:
                preTown = soup3.find('div', attrs={'aria-label': 'Town', 'class': 'media-body'}).text.replace('\n', ' ').strip()
                if preTown:
                    sample['Town'][index] = preTown
        
            except AttributeError:
                print(f"[!] Error al obtener la ciudad personal para {e}: {e}")
            try:
                prePersonalAdress = soup3.find('div', attrs={'aria-label': 'Practice addresses', 'class': 'media-body'}).text.replace('\n', ' ').strip()
                if prePersonalAdress:
                    sample['Personal Adress'][index] = prePersonalAdress
            except AttributeError:
                print(f"[!] Error al obtener la dirección personal para {e}: {e}")

[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la 

C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4246489252.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sample['Town'][index] = preTown
C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4246489252.py:19: FutureWarning: ChainedAssignm

[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la 

C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4246489252.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sample['Town'][index] = preTown
C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4246489252.py:19: FutureWarning: ChainedAssignm

[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la 

C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4246489252.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sample['Town'][index] = preTown
C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4246489252.py:19: FutureWarning: ChainedAssignm

[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la dirección personal para None: None
[!] Error al obtener la ciudad personal para None: None
[!] Error al obtener la 

C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4246489252.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sample['Town'][index] = preTown
C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4246489252.py:19: FutureWarning: ChainedAssignm

[!] Error al obtener la dirección personal para None: None


C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4246489252.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sample['Town'][index] = preTown
C:\Users\juana\AppData\Local\Temp\ipykernel_9700\4246489252.py:19: FutureWarning: ChainedAssignm

In [88]:
sample['Speciality']=sample['Speciality'].apply(lambda x:x.replace('Specialities: ',''))

In [89]:
sample['Register_as']=sample['Register_as'].apply(lambda x:x.replace('Registered as: ',''))

In [30]:
sample

,Name,Adress,Speciality,Register_as,Url,Personal Adress,Town
0,Ismaeel Bashart (01-42067),"(1.54m) Specsavers Opticians, Unit C, Kingston...",None,Optometrist,https://str.optical.org/Registrant/BD81E3D3-F0...,"Practice addresses Specsavers Opticians, Unit ...",Town Leeds
1,Deborah Hodgson (01-24911),"(1.55m) Michael Offord Optometrist, Michael Of...","Supplementary Prescribing Specialty, Independe...",Optometrist,https://str.optical.org/Registrant/123C73B1-D4...,"Practice addresses Michael Offord Optometrist,...",Town North Shields
2,Victoria Brydon (D-16182),"(1.55m) Michael Offord Optometrist, 7 Kingston...",None,Dispensing Optician,https://str.optical.org/Registrant/22278FED-D1...,"Practice addresses Michael Offord Optometrist,...",Town Newcastle upon Tyne
3,Grace Le Quelenec (D-37434),"(1.55m) Michael Offord Optometrist, 7 Kingston...",None,Dispensing Optician,https://str.optical.org/Registrant/D1BB75D4-BF...,"Practice addresses Michael Offord Optometrist,...",Town Newcastle upon Tyne
4,Michael Offord (01-8793),"(1.55m) Michael Offord Optometrist, 7 Kingston...","Supplementary Prescribing Specialty, Independe...",Optometrist,https://str.optical.org/Registrant/D4FABE6E-D7...,"Practice addresses Michael Offord Optometrist,...",Town Whitley Bay
...,...,...,...,...,...,...,...
3536,Kevin Ashforth (D-10388),"(99.89m) Asda Vision Centre, Burrigham Road, D...",None,Dispensing Optician,https://str.optical.org/Registrant/EBD2CB0D-DC...,"Practice addresses Asda Vision Centre, Burrigh...",Town Epworth
3537,Zainab Maqsood (01-42420),"(99.91m) Boots the Chemists Ltd, 1 Town Square...",None,Optometrist,https://str.optical.org/Registrant/AEF74EB3-F3...,"Practice addresses Boots the Chemists Ltd, 1 T...",Town Bury
3538,Abigail De Berry (D-9796),"(99.92m) L Woodhouse Opticians, 62-64 Highfiel...",None,Dispensing Optician,https://str.optical.org/Registrant/2BBFB33E-DC...,"Practice addresses L Woodhouse Opticians, 62-6...",Town Blackpool
3539,Stephen Woodhouse (01-17671),"(99.92m) L Woodhouse Opticians, 62-64 Highfiel...",None,Optometrist,https://str.optical.org/Registrant/638B947D-CA...,"Practice addresses L Woodhouse Opticians, 62-6...",Town Blackpool


In [31]:
sample.to_csv('stroptical_3.csv',index=False)
sample.to_excel('stroptical_3.xlsx',index=False)